##Entregable 2 - Santiago Prandini

Consigna: El script de la entrega 1 deberá adaptar datos leídos de la API y cargarlos en la tabla creada en la pre-entrega anterior en Redshift

In [33]:
#importamos las librerías que vamos a utilizar en el trabajo
import requests
import json
import pandas as pd

import os
from configparser import ConfigParser

from pathlib import Path
import sqlalchemy as sa

config = ConfigParser()

In [34]:
#nos conectamos con la carpeta drive donde se van a alojar los datos de la API correspondientes a client_id y key
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#establecemos la dirección de la carpeta donde estamos conectados
base_dir = "/content/drive/MyDrive"

In [36]:
#cambiamos al directorio actual
os.chdir(base_dir)

In [37]:
#establecemos la dirección del archivo de configuracion
config_dir = "config/config.ini"

#leemos el archivo
config.read(config_dir)

['config/config.ini']

In [38]:
#chequeamos que esté trayendo lso datos del archivo de configuración
config["credenciales_api"]["client_id"]

'e774f99f-7557-40a2-8a09-558a292e6755'

In [39]:
#definimos la variable client_id tomando los datos del archivo del drive de nombre config
client_id = config["credenciales_api"]["client_id"]

In [40]:
#definimos la variable client_secret tomando los datos del archivo del drive de nombre config
client_secret = config["credenciales_api"]["client_key"]

In [41]:
# Armamos la url con el endpoint y especificando las credenciales como parámetros. Se utiliza la API de la NASA, y el endpoint que devuelve eventos naturales, se le pasa como status el estado open ya que interesan los eventes que están aconteciendo en la actualidad
url_base = "https://eonet.gsfc.nasa.gov"
endpoint = "api/v2.1/events"

url = f"{url_base}/{endpoint}?status=open&client_id={client_id}&client_secret={client_secret}"

In [42]:
# Obtenemos datos haciendo un GET usando el método get de la librería
resp = requests.get(url)

In [43]:
#consultamos el status de la consulta
if resp.status_code == 200:
  resp_json = resp.json()
else:
  print(resp.status_code, resp.content)

In [44]:
#observamos los resultados obtenidos
print(resp_json)

{'title': 'EONET Events', 'description': 'Natural events from EONET.', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events', 'events': [{'id': 'EONET_6451', 'title': 'Tropical Cyclone Mal', 'description': '', 'link': 'https://eonet.gsfc.nasa.gov/api/v2.1/events/EONET_6451', 'categories': [{'id': 10, 'title': 'Severe Storms'}], 'sources': [{'id': 'EO', 'url': 'https://earthobservatory.nasa.gov/images/151989/hurricane-otis'}, {'id': 'JTWC', 'url': 'https://www.metoc.navy.mil/jtwc/products/sh0224.tcw'}], 'geometries': [{'date': '2023-11-13T00:00:00Z', 'type': 'Point', 'coordinates': [170.8, -9.6]}, {'date': '2023-11-13T06:00:00Z', 'type': 'Point', 'coordinates': [171.8, -11.1]}, {'date': '2023-11-13T12:00:00Z', 'type': 'Point', 'coordinates': [173.2, -12.5]}, {'date': '2023-11-13T18:00:00Z', 'type': 'Point', 'coordinates': [174, -13.5]}, {'date': '2023-11-14T00:00:00Z', 'type': 'Point', 'coordinates': [174.7, -14.7]}, {'date': '2023-11-14T06:00:00Z', 'type': 'Point', 'coordinates': [175.

In [45]:
#creamos un dataframe para ver los resultados obtenidos
df = pd.DataFrame(resp_json['events'])

In [46]:
#observamos el dataframe
df

,id,title,description,link,categories,sources,geometries
0,EONET_6451,Tropical Cyclone Mal,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 10, 'title': 'Severe Storms'}]","[{'id': 'EO', 'url': 'https://earthobservatory...","[{'date': '2023-11-13T00:00:00Z', 'type': 'Poi..."
1,EONET_6450,Wvnrp Steep Valley Fire Information,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 8, 'title': 'Wildfires'}]","[{'id': 'InciWeb', 'url': 'http://inciweb.nwcg...","[{'date': '2023-11-06T19:00:00Z', 'type': 'Poi..."
2,EONET_6449,"Ioto Volcano, Japan",,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 12, 'title': 'Volcanoes'}]","[{'id': 'SIVolcano', 'url': 'https://volcano.s...","[{'date': '2023-10-30T00:00:00Z', 'type': 'Poi..."
3,EONET_6448,Iceberg D34,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 15, 'title': 'Sea and Lake Ice'}]","[{'id': 'NATICE', 'url': 'https://usicecenter....","[{'date': '2023-10-20T00:00:00Z', 'type': 'Poi..."
4,EONET_6440,Hurricane Tammy,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 10, 'title': 'Severe Storms'}]","[{'id': 'GDACS', 'url': 'http://www.gdacs.org/...","[{'date': '2023-10-18T21:00:00Z', 'type': 'Poi..."
...,...,...,...,...,...,...,...
391,EONET_2876,Iceberg C15,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 15, 'title': 'Sea and Lake Ice'}]","[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....","[{'date': '2011-08-30T00:00:00Z', 'type': 'Poi..."
392,EONET_2878,Iceberg B09B,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 15, 'title': 'Sea and Lake Ice'}]","[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....","[{'date': '2011-08-30T00:00:00Z', 'type': 'Poi..."
393,EONET_2879,Iceberg D20A,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 15, 'title': 'Sea and Lake Ice'}]","[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....","[{'date': '2011-08-30T00:00:00Z', 'type': 'Poi..."
394,EONET_2997,Iceberg C21B,,https://eonet.gsfc.nasa.gov/api/v2.1/events/EO...,"[{'id': 15, 'title': 'Sea and Lake Ice'}]","[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....","[{'date': '2011-08-30T00:00:00Z', 'type': 'Poi..."


In [47]:
#consultamos la columnas para crear la tabla en Redshift
df.columns

Index(['id', 'title', 'description', 'link', 'categories', 'sources',
       'geometries'],
      dtype='object')

In [48]:
#instalamos el conector a redshift
%pip install "redshift_connector[full]" sqlalchemy-redshift


In [49]:
#creamos una función que arme la cadena de conexión a la base de datos a partir del archivo de configuracion

def build_conn_string(config_path, config_section):

    # Lee el archivo de configuración
    parser = ConfigParser()
    parser.read(config_path)

    # Lee la sección de configuración de PostgreSQL
    config = parser[config_section]
    host = config['host']
    port = config['port']
    dbname = config['dbname']
    username = config['username']
    pwd = config['pwd']

    # Construye la cadena de conexión
    conn_string = f'postgresql://{username}:{pwd}@{host}:{port}/{dbname}?sslmode=require'

    return conn_string

In [50]:
#creamos una función que cree la conexión a la base de datos
def connect_to_db(conn_string):

    engine = sa.create_engine(conn_string)
    conn = engine.connect()
    return conn, engine

In [51]:
#nos conectamos a la base de datos
conn_str = build_conn_string("/content/drive/MyDrive/config/config.ini", "redshift")

In [52]:
conn, engine = connect_to_db(conn_str)

In [53]:
#definimos el esquema y creamos la tabla en redshift para guardar los datos que trae la API. Definimos que si ya existe la tabla la elimine
schema = "santiprandini_coderhouse"

conn.execute(
    f"""
        DROP TABLE IF EXISTS {schema}.eventos_actuales;
        CREATE TABLE {schema}.eventos_actuales (
            id VARCHAR PRIMARY KEY,
            title VARCHAR,
            description VARCHAR,
            link VARCHAR


        );
    """
)

In [54]:
#eliminamos las columnas que vienen en formato JSON
columns_to_drop = ['categories', 'sources', 'geometries']
df = df.drop(columns=columns_to_drop)

In [55]:
#cargamos los datos obtenidos de la API en la base de datos de Redshift
df.to_sql(
    name="eventos_actuales",
    con=conn,
    schema=schema,
    if_exists="replace",
    method="multi",
    index=False,
    )

396

In [56]:
# Opcion para cargar solo registros que no existen en la tabla
df.to_sql(
    name="eventos_actuales",
    con=conn,
    schema=schema,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=1000,

)

396